# LLM Case Study 2 - Rag pipeline

### Shusrith S PES1UG22AM155
### Smera Arun Setty PES1UG22AM922

This file was used to process and clean all the tables manually and it was done separately because it took up a lot of space and time. 

# Installing libraries

In [ ]:
!pip install faiss-gpu-cu12 camelot-py pypdf2

In [ ]:
!pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 110.6 MB/s eta 0:00:00


# Download the dataset

In [ ]:
import kagglehub

path = kagglehub.dataset_download("shusrith/rag-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/rag-dataset/versions/1


# Read all tables

In [ ]:
from tabula import read_pdf

tables = read_pdf(f"{path}/combined_document_10.pdf", pages="all")

for i, table in enumerate(tables):
    print(f"\nTable {i+1}:")
    print(table)

Mar 11, 2025 7:10:00 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:03 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:03 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:03 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 11, 2025 7:10:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>




Table 1:
            Unnamed: 0  Unnamed: 1 Unnamed: 2 Unnamed: 3  Unnamed: 4  \
0  Year Ended June 30,         NaN        NaN       2016         NaN   
1        First quarter        89.0          $      4,000        43.0   
2       Second quarter        66.0        NaN      3,600        43.0   
3        Third quarter        69.0        NaN      3,600       116.0   
4       Fourth quarter        70.0        NaN      3,600        93.0   

  Unnamed: 5 Unnamed: 6   Unnamed: 7  
0        NaN       2015      2014 (a  
1          $      2,000  47  $ 1,500  
2        NaN      2,000    53  2,000  
3        NaN      5,000    47  1,791  
4        NaN      4,209    28  1,118  

Table 2:
                                           Unnamed: 0 Unnamed: 1  \
0                                                 NaN        NaN   
1                                  September 15, 2015    $  0.36   
2                                    December 2, 2015    $  0.36   
3                                      Ma

# Clean tables

The tables loaded with a lot of NaNs, `$` signs loaded as individual columns, wrong column names, and several tables consisting of subtables for particular years. All of this was cleaned individually. 

In [ ]:
tables[0].drop(columns=["Unnamed: 2", "Unnamed: 5"], inplace=True)

Unnecesary columns were dropped, columns renamed and combined columns separated. 

In [ ]:
tables[0].drop(0, inplace=True)
tables[0].columns = ["Quarter", "Shares2016", "Amount2016", "Shares2015", "Amount2015", "X"]
tables[0]["Shares2014"] = tables[0]["X"].str.split().str[0]
tables[0]["Amount2014"] = tables[0]["X"].str.split().str[1]
tables[0].drop(columns=["X"], inplace=True)
tables[0].iloc[0, 6] = 1500
tables[0]

,Quarter,Shares2016,Amount2016,Shares2015,Amount2015,Shares2014,Amount2014
1,First quarter,89.0,"4,000",43.0,"2,000",47,1500
2,Second quarter,66.0,"3,600",43.0,"2,000",53,"2,000"
3,Third quarter,69.0,"3,600",116.0,"5,000",47,"1,791"
4,Fourth quarter,70.0,"3,600",93.0,"4,209",28,"1,118"


A table containing 2015 and 2016 together was separated and added at the end

In [ ]:
tables.append(tables[1].iloc[:4])
tables.append(tables[1].iloc[-4:, :])
del tables[1]

In [ ]:
tables[2]

,Net sales,"265,595","229,234","215,639","233,715","182,795"
0,Net income,"59,531","48,351","45,687","53,394","39,510"
1,Earnings per share:,NaN,NaN,NaN,NaN,NaN
2,Basic,12.01,9.27,8.35,9.28,6.49
3,Diluted,11.91,9.21,8.31,9.22,6.45
4,Cash dividends declared per share,2.72,2.40,2.18,1.98,1.82
5,Shares used in computing earnings per share:,NaN,NaN,NaN,NaN,NaN
6,Basic,"4,955,377","5,217,242","5,470,820","5,753,421","6,085,572"
7,Diluted,"5,000,109","5,251,692","5,500,281","5,793,069","6,122,663"
8,"Total cash, cash equivalents and marketable se...","237,100","268,895","237,585","205,666","155,239"
9,Total assets,"365,725","375,319","321,686","290,345","231,839"


Column names fixed, first row appended

In [ ]:
import pandas as pd
df = pd.DataFrame([["Net sales", "265,595", "229,234", "215,639", "233,715", "182,795"]],
                  columns=["", "2018", "2017", "2016", "2015", "2014"])
tables[2].columns = ["", "2018", "2017", "2016", "2015", "2014"]
tables[2] = pd.concat([df, tables[2]], ignore_index=True)
tables[2]

,,2018,2017,2016,2015,2014
0,Net sales,"265,595","229,234","215,639","233,715","182,795"
1,Net income,"59,531","48,351","45,687","53,394","39,510"
2,Earnings per share:,NaN,NaN,NaN,NaN,NaN
3,Basic,12.01,9.27,8.35,9.28,6.49
4,Diluted,11.91,9.21,8.31,9.22,6.45
5,Cash dividends declared per share,2.72,2.40,2.18,1.98,1.82
6,Shares used in computing earnings per share:,NaN,NaN,NaN,NaN,NaN
7,Basic,"4,955,377","5,217,242","5,470,820","5,753,421","6,085,572"
8,Diluted,"5,000,109","5,251,692","5,500,281","5,793,069","6,122,663"
9,"Total cash, cash equivalents and marketable se...","237,100","268,895","237,585","205,666","155,239"


In [ ]:
df = tables[2].copy()

Subtables were split and added to the end of the tables list

In [ ]:
section_starts = df[df.iloc[:, 1:].isna().all(axis=1)].index.tolist()

# Add start & end points for splitting
section_starts.append(len(df))  # End of table

# Splitting DataFrame
dfs = []
for i in range(len(section_starts) - 1):
    start = section_starts[i] + 1
    end = section_starts[i + 1]
    dfs.append(df.iloc[start:end].reset_index(drop=True))

dfs
tables.extend(dfs)

[                                       2018  2017  2016  2015  2014
 0                              Basic  12.01  9.27  8.35  9.28  6.49
 1                            Diluted  11.91  9.21  8.31  9.22  6.45
 2  Cash dividends declared per share   2.72  2.40  2.18  1.98  1.82,
                                                            2018       2017  \
 0                                              Basic  4,955,377  5,217,242   
 1                                            Diluted  5,000,109  5,251,692   
 2  Total cash, cash equivalents and marketable se...    237,100    268,895   
 3                                       Total assets    365,725    375,319   
 4                   Non-current portion of term debt     93,735     97,207   
 5                      Other non-current liabilities     45,180     40,415   
 
         2016       2015       2014  
 0  5,470,820  5,753,421  6,085,572  
 1  5,500,281  5,793,069  6,122,663  
 2    237,585    205,666    155,239  
 3    321,686   

Unnecesary columns are dropped, columns are renamed, sub-tables are separated and added to the tables lsit.

In [ ]:
tables[3].drop(columns=["Unnamed: 0", "Unnamed: 3", "Unnamed: 6"], inplace=True)
tables[3].columns = ["Region", "2018", "Change2018", "2017", "Change2017", "2016"]
df = tables[3].copy()
section_starts = df[df.iloc[:, 1:].isna().all(axis=1)].index.tolist()
section_starts = [0, 6, 13]
# Add start & end points for splitting
section_starts.append(len(df))  # End of table

# Splitting DataFrame
dfs = []
for i in range(len(section_starts) - 1):
    start = section_starts[i] + 1
    end = section_starts[i + 1]
    dfs.append(df.iloc[start:end].reset_index(drop=True))

dfs
tables.extend(dfs)

[                 Region     2018 Change2018     2017 Change2017     2016
 0                Europe   62,420       14 %   54,938       10 %   49,952
 1         Greater China   51,942       16 %   44,764       (8)%   48,492
 2                 Japan   21,733       23 %   17,733        5 %   16,928
 3  Rest of Asia Pacific   17,407       15 %   15,199       11 %   13,654
 4       Total net sales  265,595       16 %  229,234        6 %  215,639,
                   Region     2018 Change2018     2017 Change2017     2016
 0             iPhone (1)  166,699       18 %  141,319        3 %  136,700
 1               iPad (1)   18,805       (2)%   19,222       (7)%   20,628
 2                Mac (1)   25,484       (1)%   25,850       13 %   22,831
 3           Services (2)   37,190       24 %   29,980       23 %   24,348
 4  Other Products (1)(3)   17,417       35 %   12,863       16 %   11,132
 5        Total net sales  265,595       16 %  229,234        6 %  215,639,
    Region     2018 Change201

Unnecesary columns are dropped, columns are renamed and NaN rows are removed. 

In [ ]:
tables[4].drop(columns=["Unnamed: 1", "Unnamed: 3", "Unnamed: 5"], inplace=True)
tables[4].columns = ["", "2018", "2017", "2016"]
tables[4].drop([0, 7], inplace=True)
tables[4]

,,2018,2017,2016
1,Dividends and interest income,"2,214","1,387",903
2,Interest expense,"(2,733)","(2,222)","(1,243)"
3,Net recognized gains on investments,"2,399","2,583",668
4,Net losses on derivatives,(187),(510),(443)
5,Net losses on foreign currency remeasurements,(218),(111),(129)
6,"Other, net",(59),(251),(195)
8,Total,"1,416",876,(439)


Column names are fixed, sub tables are separated and first row is fixed

In [ ]:
df = pd.DataFrame([["Revenue", "20,453", "24,090", "22,090", "23,317", "89,950"]], columns=["Quarter Ended", "september 30", "December 31", "March 31", "June 30", "Total"])
tables[5].columns = ["Quarter Ended", "september 30", "December 31", "March 31", "June 30", "Total"]
tables[5] = pd.concat([df, tables[5]], ignore_index=True)
df = tables[5].iloc[8:]
tables[5] = tables[5].iloc[:5]
tables[5]
df = df.iloc[9:]
tables.extend(df)

,Quarter Ended,september 30,December 31,March 31,June 30,Total
0,Revenue,"20,453","24,090","22,090","23,317","89,950"
1,Revenue,"20,453","24,090","22,090","23,317","89,950"
2,Gross margin,"12,609","14,189","14,030","14,861","55,689"
3,Operating income,"5,225","6,177","5,594","5,330","22,326"
4,Net income,"4,690","5,200","4,801","6,513 (c)","21,204 (c)"


Column names are fixed, combined columns are separated

In [ ]:
tables[6].columns = ["Declaration Date", "Dividend Per Share", "Record Date", "X"]
tables[6]["Total Amount"] = tables[6]["X"].str.split().str[1]
tables[6]["Payment Date"] = tables[6]["X"].str.split().str[2:]
tables[6]["Payment Date"] = tables[6]["Payment Date"].str.join(" ")
tables[6].drop(columns=["X"], inplace=True)
tables[6]

,Declaration Date,Dividend Per Share,Record Date,Total Amount,Payment Date
14,"September 16, 2014",$ 0.31,"November 20, 2014","2,547","December 11, 2014"
15,"December 3, 2014",$ 0.31,"February 19, 2015","2,532","March 12, 2015"
16,"March 10, 2015",$ 0.31,"May 21, 2015","2,496","June 11, 2015"
17,"June 9, 2015",$ 0.31,"August 20, 2015","2,475","September 10, 2015"


Column names are fixed, combined columns are separated

In [ ]:
tables[7].columns = ["Declaration Date", "Dividend Per Share", "Record Date", "X"]
tables[7].drop(0, inplace=True)
tables[7]["Total Amount"] = tables[7]["X"].str.split().str[1]
tables[7]["Payment Date"] = tables[7]["X"].str.split().str[2:]
tables[7]["Payment Date"] = tables[7]["Payment Date"].str.join(" ")
tables[7].drop(columns=["X"], inplace=True)
tables[7]

,Declaration Date,Dividend Per Share,Record Date,Total Amount,Payment Date
1,"September 15, 2015",$ 0.36,"November 19, 2015","2,868","December 10, 2015"
2,"December 2, 2015",$ 0.36,"February 18, 2016","2,842","March 10, 2016"
3,"March 15, 2016",$ 0.36,"May 19, 2016","2,821","June 9, 2016"


All processed tables are stored in csvs for use in the main file

In [ ]:
for j, i in enumerate(tables):
    try:
        i.to_csv(f"table_{j}.csv", index=False)
    except:
        pass

# Embedding strategy

Each row in each table is converted to the form of `Column1 Name : Value | Column2 Name : Value |` so that this textual form can be embedded easier.

In [ ]:
def table_to_text(row):
    return " | ".join([f"{col}: {str(row[col])}" for col in row.index])


All csvs are read and are converted to the embedding format

In [ ]:
import os
tables = []
table_text = []
l = os.listdir("data")
for i in l:
    df = pd.read_csv(f"data/{i}")
    if "Unnamed: 0" in df.columns:
        df.columns = df.columns.str.replace("Unnamed: 0", "Column1")
    tables.append(df)
    table_text.append(df.apply(table_to_text, axis=1))


Sample demo of the converted tables

In [ ]:
table_text

[0    Region: iPhone (1) | 2018: 166,699 | Change201...
 1    Region: iPad (1) | 2018: 18,805 | Change2018: ...
 2    Region: Mac (1) | 2018: 25,484 | Change2018: (...
 3    Region: Services (2) | 2018: 37,190 | Change20...
 4    Region: Other Products (1)(3) | 2018: 17,417 |...
 5    Region: Total net sales | 2018: 265,595 | Chan...
 dtype: object,
 0     Region: Americas | 2018: 112,093 | Change2018:...
 1     Region: Europe | 2018: 62,420 | Change2018: 14...
 2     Region: Greater China | 2018: 51,942 | Change2...
 3     Region: Japan | 2018: 21,733 | Change2018: 23 ...
 4     Region: Rest of Asia Pacific | 2018: 17,407 | ...
 5     Region: Total net sales | 2018: 265,595 | Chan...
 6     Region: ales by Product: | 2018: nan | Change2...
 7     Region: iPhone (1) | 2018: 166,699 | Change201...
 8     Region: iPad (1) | 2018: 18,805 | Change2018: ...
 9     Region: Mac (1) | 2018: 25,484 | Change2018: (...
 10    Region: Services (2) | 2018: 37,190 | Change20...
 11    Region: Other 

# Creating embedings
Creating vector embeddings using sentence transformers `MiniLM-L6`

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embd = []
for i in table_text:
    embd.append(model.encode(i, convert_to_numpy=True))

In [149]:
for i in embd:
    print(i.shape)

(6, 384)
(17, 384)
(3, 384)
(5, 384)
(3, 384)
(5, 384)
(4, 384)
(13, 384)
(3, 384)
(4, 384)
(7, 384)
(6, 384)


In [151]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.4 MB/s eta 0:00:00


# Indexing the tables

A new FAISS FlatL2 index is created to store the table embeddings.

In [152]:
import faiss
import numpy as np


index = faiss.IndexFlatL2(384)
table_embeddings_np = np.vstack(embd)
index.add(table_embeddings_np)
faiss.write_index(index, "table_index.faiss")


# Sample Query

In [158]:
query = "IPad in 2017"
query_embedding = model.encode(query).reshape(1, -1).astype("float32")
k = 5  # Number of results to return
distances, indices = index.search(query_embedding, k)

print("Top matching row indices:", indices[:2])
print("Corresponding distances:", distances[:2])


Top matching row indices: [[ 1 14 21 32 13]]
Corresponding distances: [[1.0067446 1.0067446 1.0531037 1.0531042 1.3381988]]


# Reverse mapping 

A `row_table_mapping` is created to map indices to the specific of a specific table. The values can be found be indexing into the tables.

In [159]:
row_table_mapping = []
row_offset = 0

for table_idx, table in enumerate(embd):
    num_rows = table.shape[0]
    for i in range(num_rows):
        row_table_mapping.append((table_idx, i))
    row_offset += num_rows
for idx in indices[0]:
    table_idx, row_idx = row_table_mapping[idx]
    print(f"Match found in Table {table_idx}, Row {row_idx}")
    print(tables[table_idx].iloc[row_idx])
    print()


Match found in Table 0, Row 1
Region        iPad (1)
2018            18,805
Change2018        (2)%
2017            19,222
Change2017        (7)%
2016            20,628
Name: 1, dtype: object

Match found in Table 1, Row 8
Region        iPad (1)
2018            18,805
Change2018        (2)%
2017            19,222
Change2017        (7)%
2016            20,628
Name: 8, dtype: object

Match found in Table 1, Row 15
Region          iPad
2018          43,535
Change2018       — %
2017          43,753
Change2017      (4)%
2016          45,590
Name: 15, dtype: object

Match found in Table 4, Row 1
Region          iPad
2018          43,535
Change2018       — %
2017          43,753
Change2017      (4)%
2016          45,590
Name: 1, dtype: object

Match found in Table 1, Row 7
Region        iPhone (1)
2018             166,699
Change2018          18 %
2017             141,319
Change2017           3 %
2016             136,700
Name: 7, dtype: object

